# Import libraries

In [16]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import os
from sklearn.model_selection import train_test_split

# Load and preprocess images

In [17]:
direction_classes = ["accelerate", "reverse", "leftTurn", "rightTurn", "leftInPlaceTurn", "rightInPlaceTurn", "stop"]

data_dir = r"C:\Users\Tomas Tomcany\PycharmProjects\rc_controller\captured_images"
file_list = os.listdir(data_dir)
image_paths = []
directions = []
pattern = "*.jpg"
for filename in file_list:
    if fnmatch.fnmatch(filename, pattern):
        image_paths.append(os.path.join(data_dir,filename))
        # The last part "zzz" of the filename xxx_yyy_zzz is the direction control
        direction = filename.split('_')[-1]
        directions.append(direction)

# Train/Test split
80/20 split is generally a good starting point.

In [18]:
X_train, X_valid, y_train, y_valid = train_test_split( image_paths, directions, test_size=0.2)
print(f"Training data: {len(X_train)}\nValidation data: {len(X_valid)}")

ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

# Image Augmentation
For artificially increasing the amount of data for training and testing. This can prevent overfitting.\
Geometric transformation – flipping, zooming, cropping, panning...\
Color space transformatios –change RGB channels\r
Kernel filtsrs blurring or sharpening an imageeage

In [ ]:
def zoom(image):
    # Specify the zoom factor
    zoom_factor = 1.5
    
    # Get the image dimensions
    height, width, _ = image.shape
    
    # Calculate the new dimensions after zooming
    new_height = int(height * zoom_factor)
    new_width = int(width * zoom_factor)
    
    # Resize the image and return
    return cv2.resize(image, (new_width, new_height))

def flip(image, direction):
    # Flip the image
    image = cv2.flip(image,1)

    # If car is making a turn flip the direction label as well
    if direction == "leftTurn":
        direction == "rightTurn"
    elif direction == "rightTurn":
        direction == "leftTurn"
    elif direction == "leftinPlaceTurn":
        direction == "rightInPlaceTurn"
    elif direction == "rightInPlaceTurn":
        direction == "leftInPlaceTurn"
   
    return image, steering_angle

def adjust_brightness(image, max_delta=30):
    # Randomly adjust the brightness of the image
    delta = np.random.uniform(-max_delta, max_delta)
    image = cv2.add(image, np.array([delta]))
    return image

def translate_shift(image, max_translation=20):
    # Randomly translate (shift) the image horizontally and/or vertically
    rows, cols, _ = image.shape
    dx = np.random.uniform(-max_translation, max_translation)
    dy = np.random.uniform(-max_translation, max_translation)
    translation_matrix = np.float32([[1, 0, dx], [0, 1, dy]])
    image = cv2.warpAffine(image, translation_matrix, (cols, rows))
    return image

def perform_random_augmentation(image, direction):
    # Randomly pick and apply augmentation to the image
    augmentations = [
        zoom, flip, adjust_brightness, translate_shift
    ]

    selected_augmentations = np.random.choice(augmentations, np.random.randint(1, len(augmentations)), replace=False)

    for augmentation in selected_augmentations:
        if augmentation == flip:
            image, direction = augmentation(image, direction)
        else:
            image = augmentation(image)

    return image, direction